In [25]:
import pandas as pd
import h5py
from config import CFG

In [27]:
import pandas as pd
from backtest_config import BT_CFG
from utils import load_calendar, weekly_fridays

# 读日行情并做 pivot
price_df = pd.read_parquet(BT_CFG.price_day_file)
if not isinstance(price_df.index, pd.MultiIndex):
    price_df = price_df.set_index(["order_book_id", "date"]).sort_index()
close_pivot = price_df["close"].unstack(0).sort_index()
avail = close_pivot.index

# 读交易周五
cal = load_calendar(BT_CFG.trading_day_file)
fridays = weekly_fridays(cal)

f0 = pd.Timestamp("2025-09-05")
# 下一个周五
f1s = fridays[fridays > f0]
print("f0 in fridays:", f0 in fridays)
print("next friday exists:", len(f1s) > 0)
if len(f1s) == 0:
    print("原因：交易日历没有比 f0 更晚的周五（可能日历没更新到该日期）。")
else:
    f1 = f1s[0]
    print("f1:", f1.date())

    i0 = avail[avail <= f0]
    i1 = avail[avail <= f1]
    print("last avail <= f0:", (i0[-1].date() if len(i0)>0 else None))
    print("last avail <= f1:", (i1[-1].date() if len(i1)>0 else None))

    if len(i0) == 0 or len(i1) == 0:
        print("原因：日行情缺少 f0 或 f1 周的数据。")
    else:
        d0 = i0[-1]; d1 = i1[-1]
        print("d0:", d0.date(), "d1:", d1.date(), "d0>=d1?", d0 >= d1)
        if d0 >= d1:
            print("原因：下一周没有比 d0 更晚的交易日（通常是下一周行情缺失或日历错位）。")

f0 in fridays: True
next friday exists: True
f1: 2025-09-12
last avail <= f0: 2025-09-05
last avail <= f1: 2025-09-05
d0: 2025-09-05 d1: 2025-09-05 d0>=d1? True
原因：下一周没有比 d0 更晚的交易日（通常是下一周行情缺失或日历错位）。


In [26]:
with h5py.File(CFG.feat_file, "r") as h5:
    dts = []
    for k in h5.keys():
        if not k.startswith("date_"): continue
        d = h5[k].attrs["date"]
        if isinstance(d, bytes): d = d.decode()
        dts.append(pd.to_datetime(d))
dts = pd.DatetimeIndex(sorted(dts))
print("H5组最近10个日期：", dts[-10:])

H5组最近10个日期： DatetimeIndex(['2025-07-04', '2025-07-11', '2025-07-18', '2025-07-25',
               '2025-08-01', '2025-08-08', '2025-08-15', '2025-08-22',
               '2025-08-29', '2025-09-05'],
              dtype='datetime64[ns]', freq=None)


In [8]:
data = pd.read_parquet('data/raw/stock_price_day.parquet').reset_index()

In [9]:
data[data['date']==pd.to_datetime('2025-09-05')]

,order_book_id,date,close,high,limit_down,limit_up,low,num_trades,open,prev_close,total_turnover,volume
3808,000001.XSHE,2025-09-05,11.72,11.74,10.57,12.91,11.63,66186.0,11.73,11.74,9.575475e+08,81966156.0
7617,000002.XSHE,2025-09-05,6.59,6.59,5.88,7.18,6.49,41833.0,6.53,6.53,7.661128e+08,117109143.0
11426,000004.XSHE,2025-09-05,10.37,10.50,9.79,10.82,9.95,3324.0,10.30,10.30,6.089208e+07,5957200.0
18711,000006.XSHE,2025-09-05,7.64,7.65,6.63,8.11,7.21,33442.0,7.37,7.37,4.074072e+08,54530038.0
22520,000007.XSHE,2025-09-05,7.50,7.50,6.68,8.16,7.29,9360.0,7.46,7.42,4.987168e+07,6732535.0
...,...,...,...,...,...,...,...,...,...,...,...,...
13169647,688799.XSHG,2025-09-05,57.88,58.25,45.69,68.53,55.93,9709.0,57.16,57.11,3.144602e+08,5510108.0
13170649,688800.XSHG,2025-09-05,73.09,73.55,56.42,84.62,69.01,32151.0,70.92,70.52,1.083755e+09,15148140.0
13171774,688819.XSHG,2025-09-05,32.50,32.55,24.22,36.34,30.42,26195.0,30.66,30.28,6.044730e+08,19031727.0
13173024,688981.XSHG,2025-09-05,114.76,114.76,91.81,137.71,114.76,0.0,114.76,114.76,0.000000e+00,0.0


In [12]:
data = pd.read_parquet('data/processed/context_features.parquet').reset_index()
data

,date,000300.XSHG_ret_1d,000300.XSHG_ret_mean_5,000300.XSHG_ret_std_5,000300.XSHG_to_mean_ratio_5,000300.XSHG_to_std_ratio_5,000300.XSHG_ret_mean_10,000300.XSHG_ret_std_10,000300.XSHG_to_mean_ratio_10,000300.XSHG_to_std_ratio_10,...,金融风格_to_mean_ratio_10,金融风格_to_std_ratio_10,金融风格_ret_mean_20,金融风格_ret_std_20,金融风格_to_mean_ratio_20,金融风格_to_std_ratio_20,金融风格_ret_mean_60,金融风格_ret_std_60,金融风格_to_mean_ratio_60,金融风格_to_std_ratio_60
0,2011-01-07,0.002209,0.006669,0.012760,0.953564,0.148123,-0.000608,0.013655,0.809473,0.200612,...,0.567726,0.183108,0.000412,0.106987,0.537206,0.156140,0.001011,0.080683,0.731714,0.325941
1,2011-01-14,-0.015733,-0.004713,0.011589,1.048595,0.113619,0.000978,0.012963,1.287362,0.306691,...,1.495571,0.448605,-0.005180,0.097558,1.407053,0.405014,-0.001110,0.076423,1.704505,0.630993
2,2011-01-21,0.013157,-0.006812,0.027285,0.879489,0.147778,-0.005762,0.019793,0.876950,0.117059,...,1.012072,0.225657,0.003285,0.092981,1.138654,0.346219,-0.002608,0.076043,1.368778,0.524628
3,2011-01-28,0.003393,0.003598,0.011334,0.921097,0.230019,-0.001607,0.020447,0.939162,0.188083,...,0.938530,0.223181,-0.000034,0.085538,1.186355,0.419410,-0.001828,0.073077,1.399843,0.539737
4,2011-02-01,0.000252,0.009281,0.006995,1.141885,0.261253,0.003418,0.016321,1.129553,0.202145,...,1.189793,0.256497,0.002004,0.085976,1.507787,0.559406,-0.000772,0.072344,1.790701,0.673112
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
746,2025-08-08,-0.002357,0.002462,0.003910,1.022472,0.079068,-0.000516,0.007198,1.202484,0.223364,...,1.305323,0.275192,0.004337,0.061703,1.417210,0.311666,0.002642,0.060523,1.208334,0.444899
747,2025-08-15,0.006959,0.004705,0.003384,0.852273,0.130228,0.003584,0.003645,0.730212,0.158350,...,0.502364,0.220821,0.010709,0.053749,0.520795,0.162933,0.006737,0.058607,0.434635,0.165774
748,2025-08-22,0.020971,0.008256,0.009159,0.876467,0.088623,0.006481,0.006774,0.765234,0.147229,...,0.913016,0.258645,0.007540,0.060156,0.745808,0.261087,0.004983,0.058854,0.636361,0.232326
749,2025-08-29,0.007388,0.005456,0.014917,0.916115,0.099943,0.006856,0.011763,0.814433,0.135040,...,1.230553,0.255178,0.001022,0.048642,0.954173,0.344503,0.004229,0.059388,0.807252,0.295211


In [30]:

data[data['order_book_id']=='688228.XSHG']
# data

,order_book_id,date,prev_close,close,num_trades,high,open,limit_down,total_turnover,limit_up,volume,low
12819946,688228.XSHG,2020-03-27,59.26,86.9120,28029.0,101.3297,96.6975,0.00,9.255352e+08,0.00,9624996.0,86.8541
12819947,688228.XSHG,2020-03-30,90.06,75.2735,11835.0,82.9939,82.9939,0.00,4.971855e+08,0.00,6243934.0,74.0576
12819948,688228.XSHG,2020-03-31,78.00,75.1963,7956.0,81.5463,76.5281,0.00,3.583822e+08,0.00,4446829.0,74.8392
12819949,688228.XSHG,2020-04-01,77.92,73.3048,6712.0,77.3966,72.3880,0.00,2.898326e+08,0.00,3784496.0,71.4905
12819950,688228.XSHG,2020-04-02,75.96,77.7633,8256.0,79.5101,74.6366,0.00,3.409392e+08,0.00,4278307.0,74.1927
...,...,...,...,...,...,...,...,...,...,...,...,...
12821264,688228.XSHG,2025-09-01,153.80,145.0100,21025.0,158.0000,146.9900,123.04,1.198278e+09,184.56,8136725.0,140.5100
12821265,688228.XSHG,2025-09-02,145.01,129.6600,16837.0,150.0000,148.5000,116.01,1.057696e+09,174.01,7737589.0,129.5700
12821266,688228.XSHG,2025-09-03,129.66,130.4000,11688.0,133.9900,127.0000,103.73,7.221273e+08,155.59,5562031.0,122.0100
12821267,688228.XSHG,2025-09-04,130.40,114.0000,12319.0,128.0000,126.8000,104.32,7.145666e+08,156.48,6021104.0,106.9900


In [28]:
stock_all = pd.read_parquet('data/processed/backtest/predictions_filtered_demo.parquet')
# stock_all = pd.read_parquet('data/processed/backtest/predictions_raw_demo.parquet')
stock = stock_all[stock_all['date']==pd.to_datetime('2025-09-05')]
stock

,date,stock,score
691922,2025-09-05,600429.XSHG,0.203025
692303,2025-09-05,600936.XSHG,0.190001
691890,2025-09-05,600382.XSHG,0.181597
691766,2025-09-05,600202.XSHG,0.180638
692067,2025-09-05,600626.XSHG,0.176273
...,...,...,...
692543,2025-09-05,601929.XSHG,-0.892757
692048,2025-09-05,600601.XSHG,-0.897472
692463,2025-09-05,601579.XSHG,-0.904562
691796,2025-09-05,600246.XSHG,-0.921938


In [19]:
stock_info = pd.read_csv('data/raw/stock_info.csv',encoding = 'gbk')
stock_info

,Unnamed: 0,code,name,name.1,ipo_date,delist_date
0,0,000001.XSHE,平安银行,PAYH,1991-04-03,2200-01-01
1,1,000002.XSHE,万科A,WKA,1991-01-29,2200-01-01
2,2,000004.XSHE,*ST国华,*STGH,1990-12-01,2200-01-01
3,3,000005.XSHE,ST星源,STXY,1990-12-10,2024-04-25
4,4,000006.XSHE,深振业A,SZYA,1992-04-27,2200-01-01
...,...,...,...,...,...,...
5436,5436,688799.XSHG,华纳药厂,HNYC,2021-07-13,2200-01-01
5437,5437,688800.XSHG,瑞可达,RKD,2021-07-22,2200-01-01
5438,5438,688819.XSHG,天能股份,TNGF,2021-01-18,2200-01-01
5439,5439,688981.XSHG,中芯国际,ZXGJ,2020-07-16,2200-01-01


In [20]:
stock.merge(stock_info,left_on='stock',right_on='code')[['date','stock','name','score']]

,date,stock,name,score
0,2025-08-29,601368.XSHG,绿城水务,0.229236
1,2025-08-29,600339.XSHG,中油工程,0.218802
2,2025-08-29,002084.XSHE,海鸥住工,0.216081
3,2025-08-29,601619.XSHG,嘉泽新能,0.213335
4,2025-08-29,600308.XSHG,华泰股份,0.210206
...,...,...,...,...
3037,2025-08-29,603716.XSHG,塞力医疗,-0.569537
3038,2025-08-29,603011.XSHG,合锻智能,-0.573074
3039,2025-08-29,600980.XSHG,北矿科技,-0.574082
3040,2025-08-29,002735.XSHE,王子新材,-0.577657
